In [3]:
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

BOTTLENECK_TENSOR_SIZE = 2048
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'

CACHE_DIR = 'D:/tensorflow/bottleneck'
INPUT_DATA = 'D:/RA3/imagenet'


TEST_PERCENTAGE = 10

STEPS = 2000
BATCH = 200

In [4]:
def create_image_lists(testing_percentage):
    result = {}
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue


        file_list = []
        dir_name = os.path.basename(sub_dir)
        file_glob = os.path.join(INPUT_DATA, dir_name, '*.' + 'JPEG')
        file_list.extend(glob.glob(file_glob))
        if not file_list: continue

        label_name = dir_name.lower()
        training_images = []
        testing_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)

            chance = np.random.randint(100)
            if chance < (testing_percentage):
                testing_images.append(base_name)
            else:
                training_images.append(base_name)

        result[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'testing': testing_images,
        }
    return result


def ImagePath(image_lists, image_dir, label_name, index, category):
    label_lists = image_lists[label_name]
    category_list = label_lists[category]
    mod_index = index % len(category_list)
    base_name = category_list[mod_index]
    sub_dir = label_lists['dir']
    full_path = os.path.join(image_dir, sub_dir, base_name)
    return full_path


def BottleneckPath(image_lists, label_name, index, category):
    return ImagePath(image_lists, CACHE_DIR, label_name, index, category) + '.txt'


def run_bottleneck_on_image(sess, image_data, image_data_tensor, BNtensor):

    bottleneck_values = sess.run(BNtensor, {image_data_tensor: image_data})

    bottleneck_values = np.squeeze(bottleneck_values)
    return bottleneck_values

def get_or_create_bottleneck(sess, image_lists, label_name, index, category, Imagetensor, BNtensor):
    label_lists = image_lists[label_name]
    sub_dir = label_lists['dir']
    sub_dir_path = os.path.join(CACHE_DIR, sub_dir)
    if not os.path.exists(sub_dir_path): os.makedirs(sub_dir_path)
    bottleneck_path = BottleneckPath(image_lists, label_name, index, category)
    if not os.path.exists(bottleneck_path):

        image_path = ImagePath(image_lists, INPUT_DATA, label_name, index, category)

        image_data = gfile.FastGFile(image_path, 'rb').read()

        bottleneck_values = run_bottleneck_on_image(sess, image_data, Imagetensor, BNtensor)

        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:

        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]

    return bottleneck_values


def get_random_cached_bottlenecks(sess, n_classes, image_lists, how_many, category, Imagetensor, BNtensor):
    bottlenecks = []
    ground_truths = []
    for _ in range(how_many):
        label_index = random.randrange(n_classes)
        label_name = list(image_lists.keys())[label_index]
        image_index = random.randrange(65536)
        bottleneck = get_or_create_bottleneck(sess, image_lists, label_name, image_index, category, Imagetensor, BNtensor)
        ground_truth = np.zeros(n_classes, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)

    return bottlenecks, ground_truths


def get_test_bottlenecks(sess, image_lists, n_classes, Imagetensor, BNtensor):
    bottlenecks = []
    ground_truths = []
    label_name_list = list(image_lists.keys())
    for label_index, label_name in enumerate(label_name_list):
        category = 'testing'
        for index, unused_base_name in enumerate(image_lists[label_name][category]):
            bottleneck = get_or_create_bottleneck(sess, image_lists, label_name, index, category,Imagetensor, BNtensor)
            ground_truth = np.zeros(n_classes, dtype=np.float32)
            ground_truth[label_index] = 1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
    return bottlenecks, ground_truths


In [5]:
image_lists = create_image_lists(TEST_PERCENTAGE)
n_classes = len(image_lists.keys())
   

In [6]:
print(n_classes)

125


In [7]:

f=gfile.FastGFile('D:/tensorflow/inception_dec_2015/tensorflow_inception_graph.pb', 'rb')
graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
BNtensor, Imagetensor = tf.import_graph_def(graph_def, return_elements=[BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME])

   


Instructions for updating:
Use tf.gfile.GFile.


In [8]:

bottleneck_input = tf.placeholder(tf.float32, [None, BOTTLENECK_TENSOR_SIZE])
ground_truth_input = tf.placeholder(tf.float32, [None, n_classes])


with tf.name_scope('final_training_ops'):
    weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, n_classes], stddev=0.001))
    biases = tf.Variable(tf.zeros([n_classes]))
    logits = tf.matmul(bottleneck_input, weights) + biases
    final_tensor = tf.nn.softmax(logits)


cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=ground_truth_input)
cross_entropy_mean = tf.reduce_mean(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy_mean)
evaluation_step = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(final_tensor, 1), tf.argmax(ground_truth_input, 1)), tf.float32))

        
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    for i in range(STEPS):

        train_bottlenecks, train_ground_truth = get_random_cached_bottlenecks(sess, n_classes, image_lists, BATCH, 'training', Imagetensor, BNtensor)
        sess.run(train_step,feed_dict={bottleneck_input: train_bottlenecks, ground_truth_input: train_ground_truth})

        if i % 15 == 0 :
            print('Step = %.1f%%' %(i/STEPS * 100))

    test_bottlenecks, test_ground_truth = get_test_bottlenecks(sess, image_lists, n_classes, Imagetensor, BNtensor)
    test_accuracy = sess.run(evaluation_step, feed_dict={bottleneck_input: test_bottlenecks, ground_truth_input: test_ground_truth})
    print('Accuracy = %.1f%%' % (test_accuracy * 100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step = 0.0%
Step = 0.8%
Step = 1.5%
Step = 2.2%
Step = 3.0%
Step = 3.8%
Step = 4.5%
Step = 5.2%
Step = 6.0%
Step = 6.8%
Step = 7.5%
Step = 8.2%
Step = 9.0%
Step = 9.8%
Step = 10.5%
Step = 11.2%
Step = 12.0%
Step = 12.8%
Step = 13.5%
Step = 14.2%
Step = 15.0%
Step = 15.8%
Step = 16.5%
Step = 17.2%
Step = 18.0%
Step = 18.8%
Step = 19.5%
Step = 20.2%
Step = 21.0%
Step = 21.8%
Step = 22.5%
Step = 23.2%
Step = 24.0%
Step = 24.8%
Step = 25.5%
Step = 26.2%
Step = 27.0%
Step = 27.8%
Step = 28.5%
Step = 29.2%
Step = 30.0%
Step = 30.8%
Step = 31.5%
Step = 32.2%
Step = 33.0%
Step = 33.8%
Step = 34.5%
Step = 35.2%
Step = 36.0%
Step = 36.8%
Step = 37.5%
Step = 38.2%
Step = 39.0%
Step = 39.8%
Step = 40.5%
Step = 41.2%
Step = 42.0%
Step = 42.